# Docling

In [1]:
import datetime
import logging
import time
from pathlib import Path
import pandas as pd

In [45]:
input_doc_paths = [
        # Path("../data/coca-cola-business-and-sustainability-report-2018.pdf"), # english
        # Path("../data/2023-saudi-aramco-sustainability-report-full-ar.pdf"), # arabic
        # Path("../data/2023-saudi-aramco-sustainability-report-full-en.pdf"), # english
        # Path("../data/20230926-rockwool-brochure-faq-bd.pdf"), # french
        # Path("../data/Scanned rockwool.pdf") # scanned/french
        # Path("../data/CC_EULA_Gen_WWCombined-MULTI-20121017_1230.pdf") # multi
        # Path("../data/websites/acciona.pdf"),
        # Path("../data/websites/adobe.pdf"),
        # Path("../data/websites/edp.pdf"),
        # Path("../data/websites/ikea.pdf"),
        # Path("../data/websites/leaseplan.pdf"),
        # Path("../data/websites/mars.pdf"),
        # Path("../data/websites/rockwool.pdf"),
        # Path("../data/websites/vestas.pdf"),
        Path("../data/websites/vfc.pdf")

    ]
output_dir = Path("../output_websites")

# Export multimodal Docling Example

In [ ]:
from docling.datamodel.base_models import AssembleOptions, ConversionStatus
from docling.datamodel.document import DocumentConversionInput
from docling.document_converter import DocumentConverter
from docling.utils.export import generate_multimodal_pages

_log = logging.getLogger(__name__)

IMAGE_RESOLUTION_SCALE = 2.0


def main():
    logging.basicConfig(level=logging.INFO)
    input_files = DocumentConversionInput.from_paths(input_doc_paths)

    # Important: For operating with page images, we must keep them, otherwise the DocumentConverter
    # will destroy them for cleaning up memory.
    # This is done by setting AssembleOptions.images_scale, which also defines the scale of images.
    # scale=1 correspond of a standard 72 DPI image
    assemble_options = AssembleOptions()
    assemble_options.images_scale = IMAGE_RESOLUTION_SCALE

    doc_converter = DocumentConverter(assemble_options=assemble_options)

    start_time = time.time()

    converted_docs = doc_converter.convert(input_files)
    success_count = 0
    failure_count = 0
    output_dir.mkdir(parents=True, exist_ok=True)
    rows = []
    for doc in converted_docs:
        if doc.status != ConversionStatus.SUCCESS:
            _log.info(f"Document {doc.input.file} failed to convert.")
            failure_count += 1
            continue
        for (
            content_text,
            content_md,
            content_dt,
            page_cells,
            page_segments,
            page,
        ) in generate_multimodal_pages(doc):

            dpi = page._default_image_scale * 72

            rows.append(
                {
                    "document": doc.input.file.name,
                    "hash": doc.input.document_hash,
                    "page_hash": page.page_hash,
                    "image": {
                        "width": page.image.width,
                        "height": page.image.height,
                        "bytes": page.image.tobytes(),
                    },
                    "cells": page_cells,
                    "contents": content_text,
                    "contents_md": content_md,
                    "contents_dt": content_dt,
                    "segments": page_segments,
                    "extra": {
                        "page_num": page.page_no + 1,
                        "width_in_points": page.size.width,
                        "height_in_points": page.size.height,
                        "dpi": dpi,
                    },
                }
            )
        success_count += 1

    print('Success count:', success_count)

    # Generate one parquet from all documents
    df = pd.json_normalize(rows)
    now = datetime.datetime.now()
    output_filename = output_dir / f"multimodal_{now:%Y-%m-%d_%H%M%S}.parquet"
    df.to_parquet(output_filename)

    end_time = time.time() - start_time

    _log.info(f"All documents were converted in {end_time:.2f} seconds.")

    if failure_count > 0:
        raise RuntimeError(
            f"The example failed converting {failure_count} on {len(input_doc_paths)}."
        )

if __name__ == "__main__":
    main()

### Load

In [ ]:
# This block demonstrates how the file can be opened with the HF datasets library
from datasets import Dataset
from PIL import Image
multimodal_df = pd.read_parquet("./scratch/multimodal_2024-09-19_125111.parquet")

# Convert pandas DataFrame to Hugging Face Dataset and load bytes into image
dataset = Dataset.from_pandas(multimodal_df)
def transforms(examples):
    examples["image"] = Image.frombytes('RGB', (examples["image.width"], examples["image.height"]), examples["image.bytes"], 'raw')
    return examples
dataset = dataset.map(transforms)

dataset

In [ ]:
print(dataset.column_names)

In [8]:
dataset[1]['image'].show()

In [ ]:
print(f"Dataset contains {len(dataset)} samples.")

# Convert a single Document

In [ ]:
from docling.document_converter import DocumentConverter

source = input_doc_paths[0]
converter = DocumentConverter()
result = converter.convert_single(source)
md = result.render_as_markdown()

with open(f"{output_dir}/markdown.md", "w") as file:
    file.write(md)

# Custom Conversion

In [4]:
import json
import logging
import time
from pathlib import Path
from typing import Iterable

from docling.backend.docling_parse_backend import DoclingParseDocumentBackend
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import ConversionStatus, PipelineOptions
from docling.datamodel.document import ConversionResult, DocumentConversionInput
from docling.document_converter import DocumentConverter
from docling.datamodel.document import DocumentLimits

_log = logging.getLogger(__name__)


def export_documents(
    conv_results: Iterable[ConversionResult],
    output_dir: Path,
    method: str
):
    output_dir.mkdir(parents=True, exist_ok=True)

    success_count = 0
    failure_count = 0

    for conv_res in conv_results:
        if conv_res.status == ConversionStatus.SUCCESS:
            success_count += 1
            doc_filename = conv_res.input.file.stem

            # Export Deep Search document JSON format:
            # with (output_dir / f"{doc_filename}.json").open("w") as fp:
            #     fp.write(json.dumps(conv_res.render_as_dict()))

            # # Export Text format:
            # with (output_dir / f"{doc_filename}.txt").open("w") as fp:
            #     fp.write(conv_res.render_as_text())

            # Export Markdown format:
            with (output_dir / f"{doc_filename}_{method}.md").open("w") as fp:
                fp.write(conv_res.render_as_markdown())

            # # Export Document Tags format:
            # with (output_dir / f"{doc_filename}.doctags").open("w") as fp:
            #     fp.write(conv_res.render_as_doctags())

        else:
            _log.info(f"Document {conv_res.input.file} failed to convert.")
            failure_count += 1

    _log.info(
        f"Processed {success_count + failure_count} docs, of which {failure_count} failed"
    )

    return success_count, failure_count

### PyPdfium without OCR

In [46]:
def main():
    logging.basicConfig(level=logging.INFO)
    pipeline_options = PipelineOptions()
    pipeline_options.do_ocr=False
    pipeline_options.do_table_structure=True
    pipeline_options.table_structure_options.do_cell_matching = False

    doc_converter = DocumentConverter(
        pipeline_options=pipeline_options,
        pdf_backend=PyPdfiumDocumentBackend,
    )
    limits = DocumentLimits(max_pages=5)
    # Define input files
    input_f = DocumentConversionInput.from_paths(input_doc_paths, limits=limits)

    start_time = time.time()

    conv_results = doc_converter.convert(input_f)
    success_count, failure_count = export_documents(
        conv_results, output_dir=output_dir, method = "pypdfium"
    )

    end_time = time.time() - start_time

    _log.info(f"All documents were converted in {end_time:.2f} seconds.")

    if failure_count > 0:
        raise RuntimeError(
            f"The example failed converting {failure_count} on {len(input_doc_paths)}."
        )

if __name__ == "__main__":
    main()

Fetching 10 files: 100%|██████████| 10/10 [00:00<00:00, 153637.51it/s]
/Users/imenekolli/Library/Caches/pypoetry/virtualenvs/pdf-parsers-JBBPt_VR-py3.11/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Processing document vfc.pdf
INFO:docling.document_converter:Finished converting page batch time=1.284
INFO:docling.document_converter:Finished converting document time-pages=1.30/1
INFO:__main__:Processed 1 docs, of which 0 failed
INFO:__main__:All documents were converted in 1.32 seconds.


### PyPdfium with OCR

In [47]:
def main():
    logging.basicConfig(level=logging.INFO)
    pipeline_options = PipelineOptions()
    pipeline_options.do_ocr=False
    pipeline_options.do_table_structure=True
    pipeline_options.table_structure_options.do_cell_matching = True

    doc_converter = DocumentConverter(
        pipeline_options=pipeline_options,
        pdf_backend=PyPdfiumDocumentBackend,
    )

    # Define input files
    input_f = DocumentConversionInput.from_paths(input_doc_paths)

    start_time = time.time()

    conv_results = doc_converter.convert(input_f)
    success_count, failure_count = export_documents(
        conv_results, output_dir=output_dir, method= "pypdfium_ocr"
    )

    end_time = time.time() - start_time

    _log.info(f"All documents were converted in {end_time:.2f} seconds.")

    if failure_count > 0:
        raise RuntimeError(
            f"The example failed converting {failure_count} on {len(input_doc_paths)}."
        )

if __name__ == "__main__":
    main()


Fetching 10 files: 100%|██████████| 10/10 [00:00<00:00, 140277.73it/s]
/Users/imenekolli/Library/Caches/pypoetry/virtualenvs/pdf-parsers-JBBPt_VR-py3.11/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Processing document vfc.pdf
INFO:docling.document_converter:Finished converting page batch time=1.252
INFO:docling.document_converter:Finished converting document time-pages=1.27/1
INFO:__main__:Processed 1 docs, of which 0 failed
INFO:__main__:All documents were converted in 1.28 seconds.


### Docling Parse without OCR

In [48]:
def main():
    logging.basicConfig(level=logging.INFO)
    pipeline_options = PipelineOptions()
    pipeline_options.do_ocr = False
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = True

    doc_converter = DocumentConverter(
        pipeline_options=pipeline_options,
        pdf_backend=DoclingParseDocumentBackend,
    )

    # Define input files
    input_f = DocumentConversionInput.from_paths(input_doc_paths)

    start_time = time.time()

    conv_results = doc_converter.convert(input_f)
    success_count, failure_count = export_documents(
        conv_results, output_dir=output_dir, method= "docling"
    )

    end_time = time.time() - start_time

    _log.info(f"All documents were converted in {end_time:.2f} seconds.")

    if failure_count > 0:
        raise RuntimeError(
            f"The example failed converting {failure_count} on {len(input_doc_paths)}."
        )


if __name__ == "__main__":
    main()

Fetching 10 files: 100%|██████████| 10/10 [00:00<00:00, 126716.13it/s]
/Users/imenekolli/Library/Caches/pypoetry/virtualenvs/pdf-parsers-JBBPt_VR-py3.11/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Processing document vfc.pdf
INFO:docling.document_converter:Finished converting page batch time=1.445
INFO:docling.document_converter:Finished converting document time-pages=1.46/1
INFO:__main__:Processed 1 docs, of which 0 failed
INFO:__main__:All documents were converted in 1.93 seconds.


### Docling Parse with OCR

In [49]:
def main():
    logging.basicConfig(level=logging.INFO)

    pipeline_options = PipelineOptions()
    pipeline_options.do_ocr=True
    pipeline_options.do_table_structure=True
    pipeline_options.table_structure_options.do_cell_matching = True

    doc_converter = DocumentConverter(
        pipeline_options=pipeline_options,
        pdf_backend=DoclingParseDocumentBackend,
    )


    # Define input files
    input_f = DocumentConversionInput.from_paths(input_doc_paths)

    start_time = time.time()

    conv_results = doc_converter.convert(input_f)
    success_count, failure_count = export_documents(
        conv_results, output_dir=output_dir, method= "docling_ocr"
    )

    end_time = time.time() - start_time

    _log.info(f"All documents were converted in {end_time:.2f} seconds.")

    if failure_count > 0:
        raise RuntimeError(
            f"The example failed converting {failure_count} on {len(input_doc_paths)}."
        )


if __name__ == "__main__":
    main()

Fetching 10 files: 100%|██████████| 10/10 [00:00<00:00, 169809.88it/s]
/Users/imenekolli/Library/Caches/pypoetry/virtualenvs/pdf-parsers-JBBPt_VR-py3.11/lib/python3.11/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please